In [1]:
import sys
import rospy
import moveit_commander

moveit_commander

<module 'moveit_commander' from '/opt/ros/melodic/lib/python2.7/dist-packages/moveit_commander/__init__.pyc'>

In [2]:

rospy.init_node('test_ur', anonymous=True)
moveit_commander.roscpp_initialize(sys.argv)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()

robot.get_group_names()

['arm', 'effector']

In [3]:
group_name = robot.get_group_names()
move_group = moveit_commander.MoveGroupCommander(group_name[0])

In [4]:
planning_frame = move_group.get_planning_frame()

In [5]:
planning_frame

'world'

In [6]:
eef_link = move_group.get_end_effector_link()

In [7]:
eef_link

'ee_link'

In [8]:

robot.get_current_state()


joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  name: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint,
  wrist_3_joint, simple_gripper_left_driver_joint, simple_gripper_left_follower_joint,
  simple_gripper_left_spring_link_joint, simple_gripper_right_driver_joint, simple_gripper_right_follower_joint,
  simple_gripper_right_spring_link_joint]
  position: [0.0005946127372586929, 0.016078054855182877, -0.0001513199664842091, 0.0008468241661532261, 4.924344995060892e-05, 0.0005710313680760137, 3.578564413686536e-07, -3.578564413686536e-07, 3.578564413686536e-07, 3.578564413686536e-07, -3.578564413686536e-07, 3.578564413686536e-07]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False

In [9]:

move_group.set_named_target("up")
plan = move_group.plan()

In [10]:
move_group.execute(plan, wait=True)

True

In [11]:

def move_joints(move_group, goal):
    move_group.go(goal, wait=True)
    move_group.stop()
    
def get_joint_state(move_group):
    joint_state = move_group.get_current_joint_values()
    print "---- rad value ----"
    print joint_state
    print "---- degree value ---- "
    print [joint*180/math.pi for joint in joint_state]

In [12]:

import math

get_joint_state(move_group)

---- rad value ----
[-0.0004658183706878205, -1.2666037409402504, 1.824032623286529, -0.7600710236102248, 3.331518585536486e-05, 4.5046842687135324e-05]
---- degree value ---- 
[-0.026689426660072613, -72.57104867135783, 104.50937100849411, -43.54886178305423, 0.0019088195432063438, 0.0025809939663626103]


In [13]:
goal = [ n * math.pi / 180 for n in [90,-100,90,-100,-90,0]]

move_joints(move_group, goal)
get_joint_state(move_group)

---- rad value ----
[1.570698467592349, -1.7452479288971023, 1.5707931817767218, -1.7452520687941906, -1.5707495860131608, 7.09595921382089e-05]
---- degree value ---- 
[89.9943930807075, -99.99534052975196, 89.9998198037321, -99.99557772838273, -89.99732195047541, 0.004065685145489067]


In [14]:
# cartesin value retrieval

def get_goal_pose(move_group):
    joint_state = move_group.get_current_pose()
    print joint_state

In [15]:

get_goal_pose(move_group)

header: 
  seq: 0
  stamp: 
    secs: 36
    nsecs: 504000000
  frame_id: "world"
pose: 
  position: 
    x: -0.109111804667
    y: 0.429623583711
    z: 0.530808906559
  orientation: 
    x: -0.405574902962
    y: 0.405674655857
    z: 0.579167261
    w: 0.579225651598


In [16]:

import geometry_msgs.msg

pose_goal = geometry_msgs.msg.Pose()

pose_goal.position.x = -0.109
pose_goal.position.y = 0.417
pose_goal.position.z = 0.618

pose_goal.orientation.x = -0.3535
pose_goal.orientation.y = 0.3535
pose_goal.orientation.z = 0.6123
pose_goal.orientation.w = 0.6124
move_group.set_pose_target(pose_goal)

In [17]:
plan = move_group.go(wait=True)
move_group.stop()
move_group.clear_pose_targets()

In [18]:

import tf

tmp = tf.transformations.euler_from_quaternion([
    pose_goal.orientation.x,
    pose_goal.orientation.y,
    pose_goal.orientation.z,
    pose_goal.orientation.w])

[each*180/math.pi for each in tmp]

[-0.008101290206148783, 59.994323938241884, 89.98596654854087]

In [19]:
tf.transformations.quaternion_from_euler(30*math.pi/180,
                                         60*math.pi/180,
                                         90*math.pi/180)

array([-0.1830127,  0.5      ,  0.5      ,  0.6830127])

In [22]:
[p,q,r] = [each*math.pi/180 for each in [0., 60, 90.]]

x,y,z,w =tf.transformations.quaternion_from_euler(p,q,r)

In [23]:
move_group.set_pose_target(pose_goal)

plan = move_group.go(wait=True)
move_group.stop()
move_group.clear_pose_targets()